## 1. 環境安裝
### 1.1 使用 anoconda (推薦)
```bash
conda env create -f environment.yml
```

安裝過程如果有錯誤，需要手動安裝缺少的 package

### 1.2 手動安裝
請參考: https://github.com/facebookresearch/maskrcnn-benchmark


## 2. 執行

### 2.1 Prepare Data
- COCO, 參考: https://github.com/facebookresearch/maskrcnn-benchmark
- VOC, 參考: https://github.com/chengchunhsu/WSIS_BBTP
- LVIS, 同上，將 json 檔從[官方網站](https://www.lvisdataset.org/dataset)下載下來後放到 ./datasets/lvis/annotations/，image 跟 COCO 一樣。

### 2.2 Train
1. 使用 8 顆 GPU
``` bash
export NGPUS=8
python -m torch.distributed.launch \
--nproc_per_node=$NGPUS tools/train_net.py \
--config-file "configs/e2e_mask_rcnn_R_50_FPN_1x.yaml" \
OUTPUT_DIR models/coco_1 SOLVER.IMS_PER_BATCH $NGPUS DATASETS.SPLIT 1
```
2. 若要使用其他數目的 GPU ，需要按照 [Detectron Scheduling Rule](https://github.com/facebookresearch/Detectron/blob/master/configs/getting_started/tutorial_1gpu_e2e_faster_rcnn_R-50-FPN.yaml#L14-L30) 調整
例如，使用四顆 GPU，要將 learning rate, batch, size 減半，scheulde 兩倍
```bash
export NGPUS=4
python -m torch.distributed.launch \
--nproc_per_node=$NGPUS tools/train_net.py \
--config-file "configs/e2e_mask_rcnn_R_50_FPN_1x.yaml" \
OUTPUT_DIR models/coco_1 SOLVER.IMS_PER_BATCH $NGPUS DATASETS.SPLIT 1 \
SOLVER.BASE_LR 0.01 SOLVER.STEPS "(120000,)" SOLVE.MAX_ITER 140000
```
3. 實驗需要用到的參數
  - 使用不同 Dataset 要加的參數
    - COCO 不同等分: `DATASETS.SPLIT n`
    - Non-VOC: `DATASETS.SPLIT 5`
    - 使用 LVIS:
    ```DATASETS.TRAIN "(\"lvis_train_cocostyle\",)" MODEL.ROI_BOX_HEAD.NUM_CLASSES 1231
    ```
    - Testing 時也是相同
  - Disable affinity loss: `MODEL.ROI_MASK_HEAD.USE_MIL_USE_AFF False `
  - Use low cost or mcg proposal: 根據 `maskrcnn_benchmark/config/paths_catalog.py`
    - 例如 `DATASETS.TRAIN "(\"coco_2017_low_cost_train\",)"
  - Disable MIL Loss and use binary cross entropy: `MODEL.ROI_MASK_HEAD.USE_MIL_LOSS False`
  - Box expand ratio: `DATASETS.EXP_RATIO 5` (可使用負數)
  - Backbone: 要使用不同的 yml: configs/e2e_mask_rcnn_R_101_FPN_1x.yaml  

### 2.3 Testing
假設 train 好的 model 放在 ./models/coco_1
last_checkpoint model_final.pth

執行以下 test.sh，參數分別為
- model 位置
- split
- way, 0 表示根據 support image 上的 class 數目
- shot, 幾個 support example
```bash
./test.sh models/coco_1 1 0 1
```

```bash
# test.sh
export NGPUS=2
echo "Positional Parameters"
dst=$1
split=${2:-1}
way=${3:-0}
shot=${4:-1}   

echo '$dst = ' $dst
echo '$split = '$split
echo '$way = '$way
echo '$shot = '$shot

python -m torch.distributed.launch --nproc_per_node=$NGPUS \
tools/test_net.py --config-file  "configs/e2e_mask_rcnn_R_50_FPN_1x.yaml" \
TEST.IMS_PER_BATCH $NGPUS OUTPUT_DIR $dst TEST.EXTRACT_FEATURE True DATASETS.SPLIT $split

python -m torch.distributed.launch --nproc_per_node=$NGPUS \
tools/test_net.py --config-file  "configs/e2e_mask_rcnn_R_50_FPN_1x.yaml" \
TEST.IMS_PER_BATCH $NGPUS TEST.USE_FEATURE False \
DATASETS.SPLIT $split OUTPUT_DIR $dst \
TEST.WAY $way TEST.SHOT $shot

```

### 2.4 常見錯誤
1. 若 apex 套件出現 "Cuda extensions are being compiled with a version of Cuda that does..." 的錯誤，可將手動安裝 apex
    - [Reference](https://zhuanlan.zhihu.com/p/80386137)
2. 因為 LVIS dataset 同一張圖的物體太多，如果在訓練中出現以下錯誤
```
OSError: [Errno 24] Too many open files
```
則在 maskrcnn_benchmark/data/datasets/coco.py 的開頭插入以下程式碼，有可能還是會錯，要多跑幾次
```python
from torch import multiprocessing
multiprocessing.set_sharing_strategy("file_system")
```
    - [Reference](https://github.com/pytorch/pytorch/issues/11201)
並且在執行時加入以下參數
```python
DATALOADER.NUM_WORKERS 1
```